# 観測のみからの模倣学習

参考：
* [Provably Efficient Imitation Learning from Observation Alone](https://arxiv.org/abs/1905.10948)
* TODO: [On the empirical estimation of integral probability metrics](https://projecteuclid.org/journals/electronic-journal-of-statistics/volume-6/issue-none/On-the-empirical-estimation-of-integral-probability-metrics/10.1214/12-EJS722.full)

一般的な模倣学習では観測と行動のペアから学習しますが、観測のみから模倣学習することも可能です。Imitation Learning from Observation Alone、略してILFOと呼びます。
今回はその手法の一つであるFAIL（Forward Adversarial Imitation Learning）を見てみましょう。

表記：
* $\left\{\mathcal{X}_h\right\}_{h=1}^H$：ステップ$h$での観測（離散）
* $c: \mathcal{X}_H \to \mathbb{R}$：コスト関数
* $\rho$：初期状態分布
* $\|f\|_\infty = \sup _{x \in \mathcal{X}}|f(x)|$
* $\|f\|_L = \sup _{x_1, x_2, x_1 \neq x_2}\left(f\left(x_1\right)-f\left(x_2\right)\right) / d\left(x_1, x_2\right)$
* $d: \mathcal{X}\times \mathcal{X} \to \mathbb{R}^+$：$\mathcal{X}$上の距離
* $\mathcal{H}$：再生核ヒルベルト空間
    * $f(x)=\langle f, k(x, \cdot) \rangle$について、$\left\langle k\left(x_1, \cdot\right), k\left(x_2, \cdot\right)\right\rangle \triangleq k\left(x_1, x_2\right)$
    * $f\in \mathcal{H}$について、$\|f\|_{\mathcal{H}}^2=\langle f, f\rangle$
* Integral probability metric (IPM)：$\sup _{f \in \mathcal{F}}\left(\mathbb{E}_{x \sim P_1}[f(x)]-\mathbb{E}_{x \sim P_2}[f(x)]\right)$
    * Total variation distanceは$\mathcal{F}=\{f: \|f\|_\infty \leq 1\}$のとき
* Linear Programming (LP) Oracle： $g\in \mathcal{G}: \min _{g \in \mathcal{G}} \sum_{i=1}^N \alpha_i g\left(x_i\right)$
    * $\mathcal{G}$がRKHSならば$\max _{g:\|g\| \leq c}\left\langle g, \sum_{i=1}^n \alpha_i \phi\left(x_i\right)\right\rangle$は解析解を持つ
* Cost Sensitive (CS) Oracle：経験コストの最小化を計算するオラクル
    * 入力：$\Pi \triangleq\{\pi: \mathcal{X} \rightarrow \Delta(\mathcal{A})\}$と$c \in \mathbb{R}^K$, i.e., $\mathcal{D}=\left\{x_i, c_i\right\}_{i=1}^N$
    * 出力：$\arg\min_\pi \sum_{i=1}^N \pi\left(\cdot \mid x_i\right)^{\top} c_i / N$
    * （証明では）Follow the Regularized Leaderを使います。[Bandit Algorithms](https://tor-lattimore.com/downloads/book/book.pdf)参照。
* 方策$\pi$の観測の分布：$\nu_{h+1}(x) \triangleq \sum_{x_h, a_h} \nu_h\left(x_h\right) \pi\left(a_h \mid x_h\right) P\left(x \mid x_h, a_h\right)$
    * $\mu^\star_{h}$：エキスパートの観測の分布


## 目的関数

まずは簡単なケースを考えてみましょう。ホライゾンは$H$までありますが、とりあえずそのうちの$h$ステップ目の方策だけ学習することを考えます。
つまり、
* $\left\{\pi_1, \ldots, \pi_{h-1}\right\}$はすでに学習済みであるとします。このとき、$h \geq 2$についての$\left\{\pi_1, \ldots, \pi_{h-1}\right\}$は分布$\nu_h \in \Delta(\mathcal{X}_h)$を決定します。
* また、エキスパートの分布から自然に$\mu_h^{\star} \in \Delta\left(\mathcal{X}_h\right)$は$h \in[H]$について決定しているとします。

このときに$\pi_h$を学習させて、$h+1$ステップ目で「方策が見る景色」が「エキスパートが見る景色」と同じになるように次の目的関数を最小化しましょう。

$$
d_{\mathcal{F}_{h+1}}\left(\pi \mid \nu_h, \mu_{h+1}^{\star}\right) 
\triangleq \max _{f \in \mathcal{F}_{h+1}}\left(\mathbb{E}_{x \sim \nu_{h+1}}[f(x)]-\mathbb{E}_{x \sim \mu_{h+1}^{\star}}[f(x)]\right)
$$

本来はこれがやりたいわけですが、実際はこの目的関数をサンプルで近似しなければなりません。次のデータセットを使ったサンプル近似を考えてみましょう。

* 方策$\pi$のデータセット：$\mathcal{D}=\left\{\left(x_h^i, a_h^i, x_{h+1}^i\right)\right\}_{i=1}^N$
    * $x_h^i \sim \nu_h, a_h^i \sim U(\mathcal{A})$, $x_{h+1}^i \sim P\left(\cdot \mid x_h^i, a_h^i\right)$
* エキスパートのデータセット：$\mathcal{D}^{\star}=\left\{\tilde{x}_{h+1}^i\right\}_{i=1}^{N^{\prime}} \stackrel{i i d}{\sim} \mu_{h+1}^{\star}$

これを踏まえて、次のようにデータセットで近似することを考えます：

$$
\widehat{d}_{\mathcal{F}_{h+1}}\left(\pi \mid \nu_h, \mu_{h+1}^{\star}\right) 
\triangleq  \max _{f \in \mathcal{F}_{h+1}}\left(\frac{1}{N} \sum_{i=1}^N \frac{\pi\left(a_h^i \mid x_h^i\right)}{1 / K} f\left(x_{h+1}^i\right)-\frac{1}{N^{\prime}} \sum_{i=1}^{N^{\prime}} f\left(\tilde{x}_{h+1}^i\right)\right)
= \max _{f \in \mathcal{F}_{h+1}} u(\pi, f)
$$
ここで効用関数$u(\pi, f) \triangleq \sum_{i=1}^N K \pi\left(a_h^i \mid x_h^i\right) f\left(x_{h+1}^i\right) / N-\sum_{i=1}^N f\left(\tilde{x}_{h+1}^i\right) / N$を導入しました。
これは$\max$のせいで普遍推定にはなりませんが、Bernsteinの不等式を使うと、それなりに良い近似であることは確かめられます。

以下、簡単のために **$N=N^\prime$である状況を考えます。**
後々のために$\widehat{d}$の集中不等式を導入しておきましょう。
いくつか表記を導入します。

* 確率変数：$v_i=K \pi\left(a_h^i \mid x_h^i\right) f\left(x_{h+1}^i\right)-f\left(\tilde{x}_{h+1}^i\right)$
    * その期待値：$v=\mathbb{E}_{x \sim \nu_h, a \sim \pi(\cdot \mid x), x^{\prime} \sim P_{x, a}}\left[f\left(x^{\prime}\right)\right]-\mathbb{E}_{x \sim \mu_{h+1}^{\star}}[f(x)]$
    * $\mathbb{E}_i v_i = v$
    * $|v_i - v|\leq 4K$
* $u(\pi, f) \triangleq \sum_{i=1}^N K \pi\left(a_h^i \mid x_h^i\right) f\left(x_{h+1}^i\right) / N-\sum_{i=1}^N f\left(\tilde{x}_{h+1}^i\right) / N=\frac{1}{N}\sum^N_{n=1}v_i$とおきます。

このとき、
$$
\begin{aligned}
\operatorname{Var}_i\left(v_i-v\right) & =\mathbb{E}_i\left(v_i-v\right)^2=\mathbb{E}_i v_i^2-v^2 \leq \mathbb{E}_i v_i^2 \\
& =\mathbb{E}_i\left(K \pi\left(a_h^i \mid x_h^i\right) f\left(x_{h+1}^i\right)-f\left(\tilde{x}_{h+1}^i\right)\right)^2 \\
& \leq \mathbb{E}_i K^2 \pi\left(a_h^i \mid x_h^i\right) f\left(x_{h+1}^i\right)-\mathbb{E}_i K \pi\left(a_h^i \mid x_h^i\right) f\left(x_{h+1}^i\right) f\left(\tilde{x}_{h+1}^i\right)+\mathbb{E}_i f\left(\tilde{x}_{h+1}^i\right)^2 \\
& \leq K+1+1 \leq 2 K
\end{aligned}
$$
が成立します。ここで最初の不等式では$|f(x)|\leq 1, \forall x, \pi(a|x)\leq 1, \forall x, a$を使っています。よって、Bernsteinの不等式から

$$
\begin{aligned}
& \left|\left(\frac{1}{N} \sum_{i=1}^N K \pi\left(a_h^i \mid x_h^i\right) f\left(x_{h+1}^i\right)-\frac{1}{N} \sum_{i=1}^N f\left(\tilde{x}_{h+1}^i\right)\right)-\left(\mathbb{E}_{\left(x, a, x^{\prime}\right) \sim \nu_h \pi P^{\star}}\left[f\left(x^{\prime}\right)\right]-\mathbb{E}_{x \sim \mu_{h+1}^{\star}}[f(x)]\right)\right| \\
& \leq 4 \sqrt{\frac{2 K \log (2|\Pi||\mathcal{F}| / \delta)}{N}}+\frac{8 K \log (2|\Pi||\mathcal{F}| / \delta)}{N} .
\end{aligned}
$$

が成立します。


### Min-Max Gameアルゴリズム

上で定義した効用関数に対して、次のアルゴリズムはそのミニマックスゲームを解くことで
$\min_{\pi \in \Pi} d_{\mathcal{F}_{h+1}}\left(\pi \mid \nu_h, \mu_{h+1}^{\star}\right)$を満たし、エキスパートの観測を模倣するような方策を探してきます。

\begin{equation}
\begin{aligned}
& \text { 1: Initialize } \pi^0 \in \Pi \\
& \text { 2: for } n=1 \text { to } T \text { do } \\
& \text { 3: } \quad f^n=\arg \max _{f \in \mathcal{F}} u\left(\pi^n, f\right) \text { (LP Oracle) } \quad \text{ \# IPMのための} f \text{を計算します}\\
& \text { 4: } \quad u^n=u\left(\pi^n, f^n\right) \quad \text{ \# } \pi_n \text{でのIPMです}\\
& \text { 5: } \quad \pi^{n+1}=\arg \min _{\pi \in \Pi} \sum_{t=1}^n u\left(\pi, f^t\right)+\phi(\pi) \text { (Regularized CS Oracle) } \quad \text{ \# IPMを最小にする方策を計算します} \\
& \text { 6: end for } \\
& \text { 7: Output: } \pi^{n^{\star}} \text { with } n^{\star}=\arg \min _{n \in[T]} u^n \quad \text{ \# 1...Tの中で最もIPMが小さくなった方策を出力します} 
\end{aligned}
\end{equation}

このMinimaxゲームについて、次の性能保証が成り立ちます。

---

$T=\Theta\left(\frac{4 K^2}{\epsilon^2}\right)$, $N=N^\prime=\Theta\left(\frac{K \log \left(\left|\Pi_h\right|\left|\mathcal{F}_{h+1}\right| / \delta\right)}{\epsilon^2}\right)$
のとき、上のMin-Maxゲームアルゴリズムは確率$1 - \delta$で次の性能保証が成立します。

$$
\left|d_{\mathcal{F}_{h+1}}\left(\pi \mid \nu_h, \mu_{h+1}^{\star}\right)-\min _{\pi^{\prime} \in \Pi_h} d_{\mathcal{F}_{h+1}}\left(\pi^{\prime} \mid \nu_h, \mu_{h+1}^{\star}\right)\right| \leq O(\epsilon) .
$$

---

これを証明してみましょう。

**証明**

まず損失関数を導入します：

* $K f^t(x, a)=K \sum_{i=1}^N \mathbf{1}\left[x=x_h^i, a=a_h^i\right] f^t\left(x_{h+1}^i\right)$
* 方策についての損失関数：$\ell_t(\pi)=(1 / N) \sum_{i=1}^N K \pi\left(a_h^i \mid x_h^i\right) f^t\left(x_{h+1}^i\right)=\langle \pi, Kf^t\rangle$
* $f$についての損失関数：$c_t(f)=(1 / N) \sum_{i=1}^N K \pi^t\left(a_h^i \mid x_h^i\right) f\left(x_{h+1}^i\right)-(1 / N) \sum_{n=1}^N f\left(\tilde{x}_{h+1}^i\right)$

このとき、$\max _\pi\|\pi\| \leq 1, \quad \max \left\|K f^t\right\| \leq K$であることに注意しましょう。
Follow the regularized leaderがno-regretを保証すること（リグレットが劣線形であること）を踏まえると、

$$
\frac{1}{T} \sum_{t=1}^T \ell_t\left(\pi^t\right)-\min _{\pi \in \Pi} \frac{1}{T} \sum_{t=1}^T \ell_t(\pi) \leq \frac{K}{\sqrt{T}}
$$

が成立します。（TODO: これはFTRLを確認しないとわからないな...でも直感的には時刻$t$で特徴が$Kf^t$のLinear banditを解いている気がするので、合ってるかも。$\sqrt{T}$リグレットになりそうだし。）

つづいて、Minimaxゲームの最適解を$\hat{\pi}^\star$と$\hat{f}^\star$と定義しましょう。
つまり、その二つは次を満たす値として定義します。

$$
\left(\hat{\pi}^{\star}, \hat{f}^{\star}\right)=\arg \min _{\pi \in \Pi} \arg \max _{f \in \mathcal{F}}\left(\frac{1}{N} \sum_{i=1}^N K \pi\left(a_h^i \mid x_h^i\right) f\left(x_{h+1}^i\right)-\frac{1}{N} \sum_{i=1}^N f\left(\tilde{x}_{h+1}^i\right)\right)
$$

さて、ここで時刻$1, \dots, T$について平均的な$\pi$と$f$についての性能保証のバウンドを導出しておきましょう。

---

$\bar{\pi}=\sum_{t=1}^T \pi^t / T$ と $\bar{f}^{\star}=\max _{f \in \mathcal{F}}\left(\mathbb{E}_{\left(x, a, x^{\prime}\right) \sim \nu_n \bar{\pi} P^{\star}}\left[f\left(x^{\prime}\right)\right]-\mathbb{E}_{x \sim \mu_{h+1}^{\star}}[f(x)]\right)$を定義します。このとき、次が成立します。

$$
\frac{1}{N} \sum_{i=1}^N K \bar{\pi}\left(a_h^i \mid x_h^i\right) \bar{f}^{\star}\left(x_{h+1}^i\right)-\frac{1}{N} \sum_{i=1}^T \bar{f}^{\star}\left(\tilde{x}_{h+1}^i\right) \leq \frac{1}{N} \sum_{i=1}^N K \hat{\pi}^{\star}\left(a_h^i \mid x_h^i\right) \hat{f}^{\star}\left(x_{h+1}^i\right)-\frac{1}{N} \sum_{i=1}^N \hat{f}^{\star}\left(\tilde{x}_{h+1}^i\right)+\frac{K}{\sqrt{T}}
$$

**証明**

$\{\pi_t\}$のno-regret学習の性質を使うと、

$$
\begin{aligned}
& \frac{1}{T} \sum_{t=1}^T\left(\frac{1}{N} \sum_{i=1}^N K \pi^t\left(a_h^i \mid x_h^i\right) f^t\left(x_{h+1}^i\right)-\frac{1}{N} \sum_{i=1}^N f^t\left(\tilde{x}_{h+1}^i\right)\right) \\
& \leq \min _{\pi \in \Pi} \frac{1}{T} \sum_{t=1}^T\left(\frac{1}{N} \sum_{i=1}^N K \pi\left(a_h^i \mid x_h^i\right) f^t\left(x_{h+1}^i\right)-\frac{1}{N} \sum_{i=1}^N f^t\left(\tilde{x}_{h+1}^i\right)\right)+\frac{K}{\sqrt{T}}
\end{aligned}
$$

が示せます。ここで、$f^t=\arg \max _{f \in \mathcal{F}} c_t(f)$なので、

$$
\begin{aligned}
& \frac{1}{N} \sum_{i=1}^N K \bar{\pi}\left(a_h^i \mid x_h^i\right) \bar{f}^{\star}\left(x_{h+1}^i\right)-\frac{1}{N} \sum_{i=1}^N \bar{f}^{\star}\left(\tilde{x}_{h+1}^i\right)=\frac{1}{T} \sum_{t=1}^T\left(\frac{1}{N} \sum_{i=1}^N K \pi^t\left(a_h^i \mid x_h^i\right) \bar{f}^{\star}\left(x_{h+1}^i\right)-\frac{1}{N} \sum_{i=1}^N \bar{f}^{\star}\left(\tilde{x}_{h+1}^i\right)\right) \\
& \leq \frac{1}{T} \sum_{t=1}^T\left(\frac{1}{N} \sum_{i=1}^N K \pi^t\left(a_h^i \mid x_h^i\right) f^t\left(x_{h+1}^i\right)-\frac{1}{N} \sum_{i=1}^N f^t\left(\tilde{x}_{h+1}^i\right)\right)
\end{aligned}
$$

が成り立ちます。
また、簡単な不等式の関係から
$$
\begin{aligned}
& \min _{\pi \in \Pi} \frac{1}{T} \sum_{t=1}^T\left(\frac{1}{N} \sum_{i=1}^N K \pi\left(a_h^i \mid x_h^i\right) f^t\left(x_{h+1}^i\right)-\frac{1}{N} \sum_{i=1}^N f^t\left(\tilde{x}_{h+1}^i\right)\right) \\
& \leq \frac{1}{T} \sum_{t=1}^T\left(\frac{1}{N} \sum_{i=1}^N K \hat{\pi}^{\star}\left(a_h^i \mid x_h^i\right) f^t\left(x_{h+1}^i\right)-\frac{1}{N} \sum_{i=1}^N f^t\left(\tilde{x}_{h+1}^i\right)\right) \\
& \leq \max _{f \in\left\{f^1, \ldots, f^T\right\}}\left(\frac{1}{N} \sum_{i=1}^N K \hat{\pi}^{\star}\left(a_h^i \mid x_h^i\right) f\left(x_{h+1}^i\right)-\frac{1}{N} \sum_{i=1}^N f\left(\tilde{x}_{h+1}^i\right)\right) \\
& \leq\left(\frac{1}{N} \sum_{i=1}^N K \hat{\pi}^{\star}\left(a_h^i \mid x_h^i\right) \hat{f}^{\star}\left(x_{h+1}^i\right)-\frac{1}{N} \sum_{i=1}^N \hat{f}^{\star}\left(\tilde{x}_{h+1}^i\right)\right)
\end{aligned}
$$
が成立します。合体して、

$$
\frac{1}{N} \sum_{i=1}^N K \bar{\pi}\left(a_h^i \mid x_h^i\right) \bar{f}^{\star}\left(x_{h+1}^i\right)-\frac{1}{N} \sum_{i=1}^N \bar{f}^{\star}\left(\tilde{x}_{h+1}^i\right) \leq \frac{1}{N} \sum_{i=1}^N K \hat{\pi}^{\star}\left(a_h^i \mid x_h^i\right) \hat{f}^{\star}\left(x_{h+1}^i\right)-\frac{1}{N} \sum_{i=1}^N \hat{f}^{\star}\left(\tilde{x}_{h+1}^i\right)+\frac{K}{\sqrt{T}}
$$
なので、証明できました。

---

最後に、いよいよ目的の定理を証明します。そのために、次の$\pi^\star$と$f^\star$を定義します。
$$
\pi^{\star}, f^{\star}=\arg \min _{\pi \in \Pi} \arg \max _{f \in \mathcal{F}}\left(\mathbb{E}_{x \sim v, a \sim \pi, x^{\prime} \sim P_{x, a}}\left[f\left(x^{\prime}\right)\right]-\mathbb{E}_{x \sim \mu_{h+1}^{\pi^{\star}}}[f(x)]\right)
$$

また、$C_N=4 \sqrt{\frac{2 K \log (2|\Pi||\mathcal{F}| / \delta)}{N}}+\frac{8 K \log (2|\Pi||\mathcal{F}| / \delta)}{N}$とします。
最初に示したBernsteinの不等式から、

$$
\begin{aligned}
& \left|\left(\frac{1}{N} \sum_{i=1}^N K \bar{\pi}\left(a_h^i \mid x_h^i\right) \bar{f}^{\star}\left(x_{h+1}^i\right)-\frac{1}{N} \sum_{i=1}^T \bar{f}^{\star}\left(\tilde{x}_{h+1}^i\right)\right)-\left(\mathbb{E}_{\left(x, a, x^{\prime}\right) \sim \nu_h \bar{\pi} P^{\star}}\left[\bar{f}^{\star}\left(x^{\prime}\right)\right]-\mathbb{E}_{x \sim \mu_{h+1}^{\star}}\left[\bar{f}^{\star}(x)\right]\right)\right| \\
& \leq \frac{1}{T} \sum_{t=1}^T\left|\left(\frac{1}{N} \sum_{i=1}^N K \pi^t\left(a_h^i \mid x_h^i\right) \bar{f}^{\star}\left(x_{h+1}^i\right)-\frac{1}{N} \sum_{i=1}^T \bar{f}^{\star}\left(\tilde{x}_{h+1}^i\right)\right)-\left(\mathbb{E}_{\left(x, a, x^{\prime}\right) \sim \nu_h \pi^t P^{\star}}\left[\bar{f}^{\star}\left(x^{\prime}\right)\right]-\mathbb{E}_{x \sim \mu_{h+1}^{\star}}\left[\bar{f}^{\star}(x)\right]\right)\right| \\
& \leq \frac{1}{T} \sum_{t=1}^T\left(C_N\right)=C_N .
\end{aligned}
$$

が成り立ちます。また、
$$
\begin{aligned}
& \left|\left(\frac{1}{N} \sum_{i=1}^N K \pi^{\star}\left(a_h^i \mid x_h^i\right) f^{\star}\left(x_{h+1}^i\right)-\frac{1}{N} \sum_{i=1}^N f^{\star}\left(\tilde{x}_{h+1}^i\right)\right)-\left(\mathbb{E}_{\left(x, a, x^{\prime}\right) \sim \nu_h \pi^{\star} P^{\star}}\left[f^{\star}\left(x^{\prime}\right)\right]-\mathbb{E}_{x \sim \mu_{h+1}^{\star}}\left[f^{\star}(x)\right]\right)\right| \\
& \leq C_N
\end{aligned}
$$
もすぐに成り立ちます（多分こっちもBernsteinで出てくる？）。

また新しい定義
$$
\hat{f}^{\prime}=\max _{f \in \mathcal{F}}\left(\frac{1}{N} \sum_{i=1}^N K \pi^{\star}\left(a_h^i \mid x_h^i\right) f\left(x_{h+1}^i\right)-\frac{1}{N} \sum_{i=1}^N f\left(\tilde{x}_{h+1}^i\right)\right)
$$
を導入します。すると、

$$
\begin{aligned}
& \max _{f \in \mathcal{F}} \mathbb{E}_{\left(x, a, x^{\prime}\right) \sim \nu_h \bar{\pi} P^{\star}}\left[f\left(x^{\prime}\right)\right]-\mathbb{E}_{x \sim \mu_{h+1}^{\star}}[f(x)]=\mathbb{E}_{\left(x, a, x^{\prime}\right) \sim \nu_h \bar{\pi} P^{\star}}\left[\bar{f}^{\star}\left(x^{\prime}\right)\right]-\mathbb{E}_{x \sim \mu_{h+1}^{\star}}\left[\bar{f}^{\star}(x)\right] \\
& \leq \frac{1}{N} \sum_{i=1}^N K \bar{\pi}\left(a_h^i \mid x_h^i\right) \bar{f}^{\star}\left(x_{h+1}^i\right)-\frac{1}{N} \sum_{i=1}^T \bar{f}^{\star}\left(\tilde{x}_{h+1}^i\right)+C_N \\
& \leq \frac{1}{N} \sum_{i=1}^N K \hat{\pi}^{\star}\left(a_h^i \mid x_h^i\right) \hat{f}^{\star}\left(x_{h+1}^i\right)-\frac{1}{N} \sum_{i=1}^N \hat{f}^{\star}\left(\tilde{x}_{h+1}^i\right)+\frac{K}{\sqrt{T}}+C_N \\
& \leq \frac{1}{N} \sum_{i=1}^N K \pi^{\star}\left(a_h^i \mid x_h^i\right) \hat{f}^{\prime}\left(x_{h+1}^i\right)-\frac{1}{N} \sum_{n=1}^N \hat{f}^{\prime}\left(\tilde{x}_{h+1}^i\right)+\frac{K}{\sqrt{T}}+C_N \\
& \leq \mathbb{E}_{\left(x, a, x^{\prime}\right) \sim \nu_h \pi^{\star} P^{\star}}\left[\hat{f}^{\prime}\left(x^{\prime}\right)\right]-\mathbb{E}_{x \sim \mu_{h+1}^{\star}}\left[\hat{f}^{\prime}(x)\right]+2 C_N+\frac{K}{\sqrt{T}} \\
& \leq \mathbb{E}_{\left(x, a, x^{\prime}\right) \sim \nu_h \pi^{\star} P^{\star}}\left[f^{\star}\left(x^{\prime}\right)\right]-\mathbb{E}_{x \sim \mu_{h+1}^{\star}}\left[f^{\star}(x)\right]+2 C_N+\frac{K}{\sqrt{T}}
\end{aligned}
$$

が成り立ちます。
つまり、

$$
\begin{aligned}
&\left(\max _{f \in \mathcal{F}} \mathbb{E}_{\left(x, a, x^{\prime}\right) \sim \nu_h \bar{\pi} P^{\star}}\left[f\left(x^{\prime}\right)\right]-\mathbb{E}_{x \sim \mu_{h+1}^{\star}}[f(x)]\right) - 
\left(\mathbb{E}_{\left(x, a, x^{\prime}\right) \sim \nu_h \pi^{\star} P^{\star}}\left[f^{\star}\left(x^{\prime}\right)\right]-\mathbb{E}_{x \sim \mu_{h+1}^{\star}}\left[f^{\star}(x)\right]\right) \\
&= 
d_{\mathcal{F}_{h+1}}\left(\bar{\pi} \mid \nu_h, \mu_{h+1}^{\star}\right)-\min _{\pi^{\prime} \in \Pi_h} d_{\mathcal{F}_{h+1}}\left(\pi^{\prime} \mid \nu_h, \mu_{h+1}^{\star}\right)\\
& \leq O(\epsilon) .
\end{aligned}
$$

また、$\pi$は$t=1, \dots, T$から一番良いものを選んでいるので、次のように平均的な方策よりも良い結果になります。
$$
d_{\mathcal{F}_{h+1}}\left(\pi \mid \nu_h, \mu_{h+1}^{\star}\right)-\min _{\pi^{\prime} \in \Pi_h} d_{\mathcal{F}_{h+1}}\left(\pi^{\prime} \mid \nu_h, \mu_{h+1}^{\star}\right)
\leq 
d_{\mathcal{F}_{h+1}}\left(\bar{\pi} \mid \nu_h, \mu_{h+1}^{\star}\right)-\min _{\pi^{\prime} \in \Pi_h} d_{\mathcal{F}_{h+1}}\left(\pi^{\prime} \mid \nu_h, \mu_{h+1}^{\star}\right)
$$

これで証明終わりです。